In [ ]:
import pandas as pd
import numpy as np

import AutoForecast as autof


%matplotlib inline  

In [ ]:
import datetime


def convert_double_to_datetime(x):
    ratio = (x - int(x))
    fulldate = datetime.datetime(int(x), 1, 1, 0, 0, 0)
    year_length = datetime.datetime(int(x) + 1, 1, 1, 0, 0, 0) - fulldate
    fulldate = fulldate + datetime.timedelta(days = int(year_length.days*ratio))
    return fulldate


In [ ]:
# examples using the autoforecast public API on some datasets (coming from R). see the link below for more datasets.
def analyzeTimeSeriesDataset(filename, horizon):
    # filename = AirPassengers.csv
    lCSVFile = "https://raw.githubusercontent.com/antoinecarme/TimeSeriesData/master/R_TSData/" + filename;
    # get the CSV file in a pandas datafrmae
    df = pd.read_csv(lCSVFile, sep=r',', engine='python');
    # in R_TSDAta , the date column is the first column
    lDateCol = df.columns[0];
    df[lDateCol] = df[lDateCol].apply(lambda x : convert_double_to_datetime(x))
    # length of the time series
    lLength = df.shape[0];
    # use only the N - H first rows for prediction, predict the H last and compare with actual values
    lTrainDataset = df[0:lLength - horizon];
    for col in df.columns:
        if(col != lDateCol):
            # forecast each cloumn separately in ths demo
            lSignalCol = col;
            # create a model(autoforecast object) ..... handle all the process
            lAutoF = autof.cAutoForecast()
            # set soem options
            lAutoF.mOptions.mEnableSeasonals = True;
            #lAutoF.mOptions.enable_slow_mode()
            #lAutoF.mOptions.mCycle_Criterion = "L2";
            #lAutoF.mOptions.mCycle_Criterion_Threshold = 10000.2;
            # train the model
            lAutoF.train(lTrainDataset , lDateCol , lSignalCol, horizon)
            # get some mdoel info
            lAutoF.getModelInfo();
            # access some advanced info ... for aficionados only!!!
            lAutoF.mSignalDecomposition.mBestTransformation.mTimeInfo.mResolution
            # define an input dataframe
            lInput = lTrainDataset.copy();
            #print(lInput.tail())
            # output dataframe ('forecast' API call ;)
            lOutput = lAutoF.forecast(lInput, horizon);
            print("Forecast Columns " , lOutput.columns);
            # Here , in the output dataframe, we keep only the data, signal and forecast outputs
            lForecastCol = lSignalCol + '_BestModelForecast';
            lForecastDataFrame = lOutput[[lDateCol , lSignalCol, lForecastCol]]
            print(lForecastDataFrame.info())
            # actual values
            print("Actual : \n" , df[[lDateCol , lSignalCol]].tail(horizon).values);
            # predcit values
            print("Predicted : \n" , lForecastDataFrame.tail(horizon).values);

            # serialize the model as json
            print("\n\n<ModelInfo>")
            print(lAutoF.to_json());
            print("</ModelInfo>\n\n")

            # serialize the forecasts as json
            print("\n\n<Forecast>")
            print(lForecastDataFrame.tail(horizon).to_json(date_format='iso'))
            print("</Forecast>\n\n")
            
            # some plots
            lAutoF.standrdPlots()
            
            lActualAndPredictedDF = pd.DataFrame();
            lActualAndPredictedDF['Date'] = df[lDateCol];
            lActualAndPredictedDF['Signal'] = df[lSignalCol];
            lActualAndPredictedDF['Forecast'] = lForecastDataFrame[lForecastCol];
            lActualAndPredictedDF['Residue'] = lActualAndPredictedDF['Forecast'] - lActualAndPredictedDF['Signal'];
            print(lActualAndPredictedDF.tail(horizon).values)
            lActualAndPredictedDF.plot.line('Date', ['Signal', 'Forecast' , 'Residue'], figsize=(32, 16))
            
    return None;


In [ ]:
R_TSeries = "AirPassengers.csv austres.csv beaver1.csv beaver2.csv BJsales.csv co2.csv DM.csv EuStockMarkets.csv \
            fdeaths.csv JohnsonJohnson.csv LakeHuron.csv ldeaths.csv lh.csv lynx.csv mdeaths.csv Nile.csv nottem.csv \
            sunspot.month.csv sunspots.csv sunspot.year.csv treering.csv UKDriverDeaths.csv UKgas.csv USAccDeaths.csv \
            WWWusage.csv".split();

analyzeTimeSeriesDataset("AirPassengers.csv" , 7)


In [ ]:
# lCSVFile = "https://raw.githubusercontent.com/antoinecarme/TimeSeriesData/master/R_TSData/" + "AirPassengers.csv";
# df = pd.read_csv(lCSVFile, sep=r',', engine='python');
# df.info()

In [ ]:
# df.Index

In [ ]:

# a = df.Index.apply(lambda x : convert_double_to_datetime(x))
# a
